In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# groupby 스킬업
pd.options.display.float_format = '{:.2f}'.format # 소수점 출력옵션
data = [['김판다', 'A', '남', 95, 77], ['송중기', 'B', '남', 93, 92], 
        ['김나현', 'B', '여', 88, 60], ['박효신', 'A', '남', 85, 83], 
        ['강승주', 'B', '여', 78, 92], ['권보아', 'A', '여', 72, 75]]
df = pd.DataFrame(data, columns=['이름', '반', '성별', '국어', '수학'])
df

,이름,반,성별,국어,수학
0,김판다,A,남,95,77
1,송중기,B,남,93,92
2,김나현,B,여,88,60
3,박효신,A,남,85,83
4,강승주,B,여,78,92
5,권보아,A,여,72,75


In [3]:
# groupby 집계함수
df.groupby(['반', '성별'])['국어'].mean()

반  성별
A  남    90.00
   여    72.00
B  남    93.00
   여    83.00
Name: 국어, dtype: float64

In [4]:
df.groupby(['반', '성별'])[['국어', '수학']].mean()

국어    수학
반 성별            
A 남  90.00 80.00
  여  72.00 75.00
B 남  93.00 92.00
  여  83.00 76.00

In [5]:
# 인덱싱을 하지 않으면 전체에 적용한다 (단 by에 지정된 열은 제외)
df.groupby(['반', '성별']).mean(numeric_only=True)

국어    수학
반 성별            
A 남  90.00 80.00
  여  72.00 75.00
B 남  93.00 92.00
  여  83.00 76.00

In [6]:
# groupby 스킬업2
df.groupby('반')['국어'].agg(['mean', 'std'])

,mean,std
반,,
A,84.00,11.53
B,86.33,7.64


In [7]:
# 함수를 튜플로 입력해 열 이름을 바꿀 수 있다
df.groupby('반')['국어'].agg([('국어평균', 'mean'), ('표준편차', 'std')])

,국어평균,표준편차
반,,
A,84.00,11.53
B,86.33,7.64


In [8]:
# 열마다 다른 집계함수 적용
df.groupby('반').agg({'국어':'mean', '수학':'count'})

,국어,수학
반,,
A,84.00,3
B,86.33,3


In [9]:
# lambda 함수도 사용 가능하다.
df.groupby('반')['수학'].agg(lambda x: (x>80).sum())

반
A    1
B    2
Name: 수학, dtype: int64

In [10]:
# groupby 스킬업3
# transform은 집계함수의 적용 결과를 원본 df와 같은 길이의 시리즈로 반환한다
df.groupby('반')['수학'].transform('mean')

0   78.33
1   81.33
2   81.33
3   78.33
4   81.33
5   78.33
Name: 수학, dtype: float64

In [11]:
# 필터링할 때 사용
cond1 = df['수학'] > df.groupby('반')['수학'].transform('mean')
df[cond1]

,이름,반,성별,국어,수학
1,송중기,B,남,93,92
3,박효신,A,남,85,83
4,강승주,B,여,78,92


In [12]:
# apply 스킬업
data = [['82점', '81점', '77점'], ['91점', '95점', '83점'], 
         ['78점', '72점', '88점'], ['82점', '87점', '72점']]
s1 = pd.Series(['80점', '75점', '77점', '60점'])
s2 = pd.Series(['4등', '3등', '1등', '2등'], index=list('ABCD'))
s3 = pd.Series(['없음', '사오기', '만원', '2만원'], 
               index=['1등', '2등', '3등', '4등'])
df = pd.DataFrame(data, index=list('ABCD'), columns=['국어', '영어', '수학'])
df

,국어,영어,수학
A,82점,81점,77점
B,91점,95점,83점
C,78점,72점,88점
D,82점,87점,72점


In [13]:
# 시리즈에 apply 적용하기
x = s1[0]
int(x[:-1])
s1.apply(lambda x: int(x[:-1]))

0    80
1    75
2    77
3    60
dtype: int64

In [14]:
# map 함수는 시리즈에 적용할 때 apply와 유사하다
s1.map(lambda x: int(x[:-1]))

0    80
1    75
2    77
3    60
dtype: int64

In [15]:
print(s2)
print(s3)

A    4등
B    3등
C    1등
D    2등
dtype: object
1등     없음
2등    사오기
3등     만원
4등    2만원
dtype: object


In [16]:
# 시리즈 형식의 매퍼를 입력받을 수 있음
s2.map(s3)

A    2만원
B     만원
C     없음
D    사오기
dtype: object

In [17]:
s2.apply(lambda x: s3[x])

A    2만원
B     만원
C     없음
D    사오기
dtype: object

In [18]:
# apply와 applymap의 차이
df

,국어,영어,수학
A,82점,81점,77점
B,91점,95점,83점
C,78점,72점,88점
D,82점,87점,72점


In [19]:
# applymap has been deprecated
df.applymap(lambda x: int(x[:-1]))

,국어,영어,수학
A,82,81,77
B,91,95,83
C,78,72,88
D,82,87,72


In [20]:
df.apply(lambda x: x.map(lambda y: int(y[:-1])))

,국어,영어,수학
A,82,81,77
B,91,95,83
C,78,72,88
D,82,87,72


In [21]:
# 데이터 구조 바꾸기 스킬업
pd.options.display.max_rows = 6 # 판다스 버전업에 따라 6행만 출력의 바뀐 코드
df1 = pd.DataFrame([[95, ['김판다', '강승주']], [92, '송중기']], 
                   columns=['점수', '이름'])
df2 = pd.DataFrame([[95, '김판다/강승주'], [92, '송중기']], 
                   columns=['점수', '이름'])

In [22]:
# explode
df1

,점수,이름
0,95,"[김판다, 강승주]"
1,92,송중기


In [23]:
df1.explode('이름', ignore_index=True)

,점수,이름
0,95,김판다
1,95,강승주
2,92,송중기


In [24]:
# 리스트가 있는 이유?
df2

,점수,이름
0,95,김판다/강승주
1,92,송중기


In [25]:
df2['이름'] = df2['이름'].str.split('/')
df2.explode('이름', ignore_index=True)

,점수,이름
0,95,김판다
1,95,강승주
2,92,송중기


In [26]:
# explode 함수 실습하기
url = 'https://raw.githubusercontent.com/panda-kim/csv_files/main/naver.csv'
# 데이터가 너무 클때 일단 행 몇개만 가져오고 확인하기
# pd.read_csv(url, nrows=2)
df_ex1 = pd.read_csv(url, usecols=['title', 'author'])
df_ex1

,title,author
0,가난을 등에 업은 소녀,B급달궁 / 오은지
1,가담항설,랑또
2,가령의 정체불명 이야기,가령
...,...,...
1853,[드라마원작] 한번 더 해요,미티 / 구구
1854,[드라마원작] 내 ID는 ...,기맹기
1855,[드라마원작] 지금 우리 ...,주동근


In [27]:
df_ex2 = df_ex1.copy()
df_ex2['author'] = df_ex2['author'].str.split(' / ')
df_ex2

,title,author
0,가난을 등에 업은 소녀,"[B급달궁, 오은지]"
1,가담항설,[랑또]
2,가령의 정체불명 이야기,[가령]
...,...,...
1853,[드라마원작] 한번 더 해요,"[미티, 구구]"
1854,[드라마원작] 내 ID는 ...,[기맹기]
1855,[드라마원작] 지금 우리 ...,[주동근]


In [28]:
df_ex2.explode('author')

,title,author
0,가난을 등에 업은 소녀,B급달궁
0,가난을 등에 업은 소녀,오은지
1,가담항설,랑또
...,...,...
1853,[드라마원작] 한번 더 해요,구구
1854,[드라마원작] 내 ID는 ...,기맹기
1855,[드라마원작] 지금 우리 ...,주동근


In [29]:
s = pd.Series(['a', 'bc'])
s

0     a
1    bc
dtype: object

In [30]:
'/'.join(s)

'a/bc'

In [31]:
# df_ex2.explode('author').groupby('author')['title'].agg('/'.join).reset_index()
# as_index=False는 그룹을 나누는 key가 열이 되어 데이터 프레임이 된다.
df_ex2.explode('author').groupby('author', as_index=False)['title'].agg('/'.join)

,author,title
0,-2℃,헬 인 파라다이스
1,0환이,가짜인간
2,12B,뱀피르
...,...,...
1737,히디,시월드 판타지
1738,히어리,꽃미남 저승사자/재혼 황후/하렘의 남자들
1739,힐링달,인싸라이프


In [32]:
# 시각화 스킬업
# pyecharts
# pip install pyecharts==1.9.1
# pip install pandasecharts==0.5 # 2019년 이후로 업데이트 멈춤

In [33]:
import pandas as pd
from pyecharts.globals import ThemeType # 테마 바꿀 때 사용
from pyecharts.charts import Timeline # 여러 그래프를 순차적으로 보여주게 만들 때 사용
from pandasecharts import echart # 그래프를 그려줄 pandasecharts
import IPython # 구글 코랩에서 그래프를 출력할 때 사용

In [34]:
data1 = [['강승주', 98, 72], ['김판다', 89, 91], 
         ['박효신', 77, 78], ['손승연', 62, 93]]
data2 = [['강승주', 72, 79], ['김판다', 83, 81], 
         ['박효신', 74, 87], ['손승연', 92, 89]]
df1 = pd.DataFrame(data1, columns=['이름', '국어', '영어'])
df2 = pd.DataFrame(data2, columns=['이름', '국어', '영어'])

In [35]:
df1.echart.bar(x='이름', ys=['국어','영어'], sort='국어',
               figsize=(600, 400), title='중간고사', subtitle='결과',
              theme=ThemeType.DARK).render() # sort는 다른 열도 가능
IPython.display.HTML(filename='render.html')

In [36]:
# 그래프 2개 그리기, Timeline 사용
tl = Timeline({'width':'600px', 'height':'400px'})
bar1 = df1.echart.bar(x='이름', ys=['국어', '영어'], title='1학기', subtitle='중간고사')
bar2 = df2.echart.bar(x='이름', ys=['국어', '영어'], title='1학기', subtitle='기말고사')
tl.add(bar1, '중간고사').add(bar2, '기말고사').render()
IPython.display.HTML(filename='render.html')